In [3]:
from numpy import *
from scipy.optimize import fsolve


In [5]:
# set up constants
bigg = (6.67384*10**-11)*((149.597871*10**9)**-3)*(1.98855*10**30)*((365.24*24.*3600.)**2)
mass = 4.31*10**6 #smbh mass
degtorad = pi/180.
arcsectoau = degtorad*(1/3600.)*(2.06264806*10**8)*8.33
time = linspace(0,1000,10000)
speedoflight = ((pi*10**7)/(150.*10**9.))*3.*10**8


In [ ]:
#initialize data structures for radius and true anomaly:
trueanomalymatrix=zeros((len(time),n))
radiusmatrix=zeros((len(time),n))
ellipse_xmatrix=zeros((len(time),n))
ellipse_ymatrix=zeros((len(time),n))
ellipse_zmatrix=zeros((len(time),n))

for j in range(0,n):
    inp = ([inc[j]*degtorad,asc[j]*degtorad,sma[j]*arcsectoau,
            ecc[j],argp[j]*degtorad])


In [ ]:
meananomep = sqrt((bigg*(mass+15))/(inp[2]**3))*2000*degtorad #computes M(0)
inp.append(meananomep) #inp[5]
period = 2*pi*sqrt((inp[2]**3)/(bigg*mass)) #does what it says on the tin
inp.append(period) #inp[6]

# comment to remove precession effect
precessionperperiod=(6.*pi*bigg*(mass+15.))/((speedoflight**2)*inp[2]*(1.-inp[3]**2))

In [ ]:
def meananomaly(ma,T,time):
    #computes the mean anomaly as a function of time
    return mod((ma+((2*pi)/T)*mod(time,T)),2*pi)
meananom = meananomaly(inp[5],inp[6],time)

In [ ]:
eccenanom = []
for i in meananom:
    #computes eccentric anomaly from the mean anomaly
    def func(ea,i,ec):
        return ea - ec*sin(ea) - i
    sol = fsolve(func,0,args=(i, inp[3]))
    eccenanom.append(sol)


In [ ]:
eccenanom = []
for i in meananom:
    #computes eccentric anomaly from the mean anomaly
    def func(ea,i,ec):
        return ea - ec*sin(ea) - i
    sol = fsolve(func,0,args=(i, inp[3]))
    eccenanom.append(sol)


In [ ]:
trueanom = []
for i in eccenanom:
    #computes the true anomaly as a function of time

    sol2 =-2*(arctan2(sqrt(1-inp[3])*cos(i/2),sqrt(1+inp[3])*sin(i/2)))+pi
    trueanom.append(sol2)
for i in range(0,len(trueanom)):
    trueanomalymatrix[i][j]=trueanom[i]
#plot(time,trueanom) to check (good)


In [ ]:
def ellipseinplane(a,e,th):
    #computes the shape of the ellipse
    return (a*(1-e**2))/(1+e*cos(th))
r = []
for i in trueanom:
    #computes the position of orbiting body as a function of time
    sol4 = ellipseinplane(inp[2],inp[3],i)
    r.append(sol4)


In [ ]:
for i in range(0,len(r)):
    radiusmatrix[i][j]=r[i]
def ellipseinspacex(rad,trueanomaly,incl,ascn,argpe):
    return rad*sin(trueanomaly)*(-cos(ascn)*sin(argpe)-...
                                 cos(incl)*cos(argpe)*sin(ascn)) +
    rad*cos(trueanomaly)*(cos(argpe)*cos(ascn)-
                          cos(incl)*sin(argpe)*sin(ascn))
def ellipseinspacey(rad,trueanomaly,incl,ascn,argpe):
    return -rad*cos(trueanomaly)*(cos(incl)*cos(ascn)*sin(argpe)+
                                      cos(argpe)*sin(ascn)) -
    rad*sin(trueanomaly)*(cos(incl)*cos(argpe)*cos(ascn) -
                          sin(argpe)*sin(ascn))

def ellipseinspacez(rad,trueanomaly,incl,ascn,argpe):
    return rad*cos(argpe)*sin(trueanomaly)*sin(incl) +
    rad*cos(trueanomaly)*sin(incl)*sin(argpe)

ellipse_x = []
for i in range(0,len(r)):
    sol5 = ellipseinspacex(r[i],trueanom[i],inp[0],inp[1],(inp[4]...
                                                           #break here to remove precession in each coordinate
                                                           +i*(precessionperperiod/(period/0.1))))
ellipse_x.append(sol5)
for i in range(0,len(r)):
    ellipse_xmatrix[i][j]=ellipse_x[i]
ellipse_y = []
for i in range(0,len(r)):
    sol5 = ellipseinspacey(r[i],trueanom[i],inp[0],inp[1],(inp[4]
                                                           +i*(precessionperperiod/(period/0.1))))
ellipse_y.append(sol5)
for i in range(0,len(r)):
    ellipse_ymatrix[i][j]=ellipse_y[i]
ellipse_z = []
for i in range(0,len(r)):
    sol5 = ellipseinspacez(r[i],trueanom[i],inp[0],inp[1],(inp[4]
                                                           +i*(precessionperperiod/(period/0.1))))
ellipse_z.append(sol5)
for i in range(0,len(r)):
    ellipse_zmatrix[i][j]=ellipse_z[i]


In [ ]:
# plots time series distance betweens tars
distx = abs(ellipse_xmatrix[:,0]-ellipse_xmatrix[:,2])
disty = abs(ellipse_ymatrix[:,0]-ellipse_ymatrix[:,2])
distz = abs(ellipse_zmatrix[:,0]-ellipse_zmatrix[:,2])
normvec = []
for i in range(0,len(time)):
    dist = sqrt((distx[i])**2+(disty[i])**2+(distz[i])**2)
normvec.append(dist)

In [ ]:
fig = plt.figure()
plt.plot(time,normvec)
plt.xlabel(’t (yr)’)
plt.ylabel(’r (AU)’)
# <codecell>
#plots trajectory in plane of sky
fig2 = plt.figure()
plt.plot(ellipse_ymatrix[:,3],ellipse_xmatrix[:,3],’b’)
plt.xlabel(’r (AU)’)
plt.ylabel(’r (AU)’)
print "done"
# <codecell>
#histogram for close approaches.
fig2 = plt.figure()
plt.hist(dataminlist,bins=50,color=’blue’)
fig2.suptitle(’Minimum separations of stars in S-population’)
plt.xlabel(’Time (Yrs from Epoch2000)’)
plt.ylabel(’Number of objects’)

In [ ]:
from random import uniform

#randomly generates orbital parameters from a flat distribution.
incsim = []
smasim = []
eccsim = []
argpsim = []
ascsim = []
for i in range(0,1000): #choose number of objects to simulate
    incsim.append(uniform(min(inc),max(inc)))
smasim.append(uniform(min(sma),max(sma)))
eccsim.append(uniform(min(ecc),max(ecc)))
argpsim.append(uniform(min(argp),max(argp)))
ascsim.append(uniform(min(asc),max(asc)))

In [7]:
# 2 body script
star1positionxn=[]
star1positionyn=[]
star2positionxn=[]
star2positionyn=[]
# <codecell>
countn = 0
dtn = .001
# initial positions
pos2n = star_1[0]
pos3n = star_2[0]
star1positionxn.append(pos2n[0])
star1positionyn.append(pos2n[1])
star2positionxn.append(pos3n[0])
star2positionyn.append(pos3n[1])
# initial velocities
vel2n = velocity_star1[0]
vel3n = velocity_star2[0]
while countn<1000000:
#star1
ft2n = force(pos1n,pos2n,4.31*10**6)
pos2n = pos2n+(vel2n*dtn)+((ft2n/2.)*dtn**2)
ftdt2n = force(pos1n,pos2n,4.31*10**6)
vel2n = vel2n+((dtn/2.)*(ft2n+ftdt2n))
star1positionxn.append(pos2n[0])
star1positionyn.append(pos2n[1])
#star2
ft3n = force(pos1n,pos3n,4.31*10**6)
pos3n = pos3n+(vel3n*dtn)+(ft3n/2.)*dtn**2
ftdt3n = force(pos1n,pos3n,4.31*10**6)
vel3n = vel3n+(dtn/2.)*(ft3n+ftdt3n)
star2positionxn.append(pos3n[0])
star2positionyn.append(pos3n[1])
countn = countn+1
# 3 body script
star1positionx=[]
star1positiony=[]
star2positionx=[]
star2positiony=[]
# <codecell>
count = 0
dt = .001
# initial positions
pos2 = star_1[0]
pos3 = star_2[0]
star1positionx.append(pos2[0])
star1positiony.append(pos2[1])
star2positionx.append(pos3[0])
star2positiony.append(pos3[1])
# initial velocities
vel2 = velocity_star1[0]
vel3 = velocity_star2[0]


while count<1000000:
#star1
ft2 = force(pos1,pos2,4.31*10**6)+force(pos3,pos2,15.)
pos2 = pos2+(vel2*dt)+((ft2/2.)*dt**2)
ftdt2 = force(pos1,pos2,4.31*10**6)+force(pos3,pos2,15.)
vel2 = vel2+((dt/2.)*(ft2+ftdt2))
star1positionx.append(pos2[0])
star1positiony.append(pos2[1])
#star2
ft3 = force(pos1,pos3,4.31*10**6)+force(pos2,pos3,15.)
pos3 = pos3+(vel3*dt)+(ft3/2.)*dt**2
ftdt3 = force(pos1,pos3,4.31*10**6)+force(pos2,pos3,15.)
vel3 = vel3+(dt/2.)*(ft3+ftdt3)
star2positionx.append(pos3[0])
star2positiony.append(pos3[1])
count = count+1


posx1 = asarray(star1positionx)
posx1n = asarray(star1positionxn)
posy1 = asarray(star1positiony)
posy1n = asarray(star1positionyn)
posx2 = asarray(star2positionx)
posx2n = asarray(star2positionxn)
posy2 = asarray(star2positiony)
posy2n = asarray(star2positionyn)
st1mod = sqrt((posx1-posx1n)**2+(posy1-posy1n)**2)
for i in st1mod:
    if i>50:
    print ’star 1 detected’
break

    st2mod = sqrt((posx2-posx2n)**2+(posy2-posy2n)**2)
    for i in st2mod:
        if i>50:
        print ’star 2 detected’
    break


IndentationError: expected an indented block (1042141977.py, line 21)